In [1]:
# Notebook: 5 - Preparação dos Dados para Reinforcement Learning (RL)

import pandas as pd
import numpy as np
import os

# Caminho base dos arquivos CSV
caminho_base = r"c:\\Users\\wilso\\OneDrive\\MBA FIAP\\10. Reinforcement Learning\\ENTREGA DO EXERCÍCIO\\"

# Lista de tickers dos ativos e benchmarks
tickers = ['vale3', 'petr4', 'brfs3', 'bova11', 'imab11']

# Função para carregar todos os dados
def carregar_dados(tickers):
    dados = {}
    for ticker in tickers:
        arquivo = f"dados_{ticker}.csv"
        caminho = os.path.join(caminho_base, arquivo)
        try:
            df = pd.read_csv(caminho, index_col=0, parse_dates=True)
            dados[ticker] = df
            print(f"\u2705 {ticker.upper()} carregado com {len(df)} registros")
        except Exception as e:
            print(f"❌ Erro ao carregar {ticker}: {e}")
    return dados

# 5.1 Limpeza e Alinhamento Temporal

def alinhar_dados(dados):
    dfs = [df[['Close']].rename(columns={'Close': ticker}) for ticker, df in dados.items() if 'Close' in df.columns]
    df_merged = pd.concat(dfs, axis=1, join='inner')
    df_merged.dropna(inplace=True)
    print(f"\n\u2705 Datas comuns alinhadas: {df_merged.index.min().date()} a {df_merged.index.max().date()}")
    return df_merged

# 5.2 Cálculo dos Indicadores Técnicos

def calcular_indicadores(df, ticker):
    df = df.copy()
    df['SMA_10'] = df['Close'].rolling(window=10).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI_14'] = 100 - (100 / (1 + rs))
    df['BB_MID'] = df['Close'].rolling(window=20).mean()
    df['BB_STD'] = df['Close'].rolling(window=20).std()
    df['BB_UPPER'] = df['BB_MID'] + 2 * df['BB_STD']
    df['BB_LOWER'] = df['BB_MID'] - 2 * df['BB_STD']
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_SIGNAL'] = df['MACD'].ewm(span=9, adjust=False).mean()
    return df.dropna()

# 5.3 Montagem do Espaço de Estados

def construir_estado(df_indicadores, posicao=0, saldo=100000):
    df_estado = df_indicadores.copy()
    df_estado['posicao'] = posicao
    df_estado['saldo'] = saldo
    return df_estado

# Execução sequencial

# Carregar dados brutos
dados_brutos = carregar_dados(tickers)

# Alinhar datas com dados completos
df_alinhado = alinhar_dados(dados_brutos)

# Calcular indicadores para cada ativo
indicadores = {}
for ticker in ['vale3', 'petr4', 'brfs3']:
    if ticker in dados_brutos:
        indicadores[ticker] = calcular_indicadores(dados_brutos[ticker], ticker)

# Exemplo de montagem de estado para um ativo
estado_exemplo = construir_estado(indicadores['vale3'])

# Visualização inicial
estado_exemplo.tail()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\wilso\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\wilso\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\wilso\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

AttributeError: _ARRAY_API not found

C:\Users\wilso\AppData\Local\Temp\ipykernel_12924\3616054807.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(caminho, index_col=0, parse_dates=True)
C:\Users\wilso\AppData\Local\Temp\ipykernel_12924\3616054807.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(caminho, index_col=0, parse_dates=True)
C:\Users\wilso\AppData\Local\Temp\ipykernel_12924\3616054807.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(caminho, index_col=0, parse_dates=True)
C:\Users\wilso\AppData\Local\Temp\ipykernel_12924\3616054807.py:20:

✅ VALE3 carregado com 1246 registros
✅ PETR4 carregado com 1246 registros
✅ BRFS3 carregado com 1246 registros
✅ BOVA11 carregado com 1246 registros
✅ IMAB11 carregado com 1246 registros


AttributeError: 'str' object has no attribute 'date'